# Introduction

**Related notebooks**

Utility script notebook (with addtional functions, aggregators; data collection):

https://www.kaggle.com/andreynesterov/home-credit-baseline-data

Training models notebooks:

https://www.kaggle.com/andreynesterov/home-credit-baseline-training

https://www.kaggle.com/code/andreynesterov/home-credit-baseline-training-no-dates

https://www.kaggle.com/code/andreynesterov/home-credit-baseline-training-lightautoml

# Dependencies

In [1]:
!pip install --no-index -Uq --find-links=/kaggle/input/lightautoml-038-dependencies pandas==2.0.3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.0.11 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2024.1.0 which is incompatible.
cuml 23.

In [2]:
import os
import gc
from glob import glob
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import polars as pl

from sklearn.metrics import roc_auc_score, log_loss
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
from scipy.optimize import minimize

import joblib
import lightgbm as lgb
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


# Predictions

In [3]:
import home_credit_baseline_data as data_nb

In [4]:
### from https://www.kaggle.com/code/batprem/home-credit-risk-mode-utility-scripts
# 評価スコアを出すための関数
def gini_stability(base, score_col="score", w_fallingrate=88.0, w_resstd=-0.5):
    gini_in_time = base.loc[:, ["WEEK_NUM", "target", score_col]]\
        .sort_values("WEEK_NUM")\
        .groupby("WEEK_NUM")[["target", score_col]]\
        .apply(lambda x: 2*roc_auc_score(x["target"], x[score_col])-1).tolist()
    
    x = np.arange(len(gini_in_time))
    y = gini_in_time
    a, b = np.polyfit(x, y, 1)
    y_hat = a*x + b
    residuals = y - y_hat
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_in_time)
    return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

In [5]:
def predict_proba_in_batches(model, data, batch_size=100000, predict_mode="base"):
    num_samples = len(data)
    num_batches = int(np.ceil(num_samples / batch_size))
    probabilities = np.zeros((num_samples,))

    for batch_idx in range(num_batches):
        print(f"Processing batch: {batch_idx+1}/{num_batches}")
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, num_samples)
        X_batch = data.iloc[start_idx:end_idx]
        if predict_mode == "base":
            batch_probs = model.predict_proba(X_batch)[:, 1]
        elif predict_mode == "lightautoml":
            batch_probs = model.predict(X_batch).data.squeeze()
        probabilities[start_idx:end_idx] = batch_probs
        gc.collect()

    return probabilities

In [6]:
train_base_df = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_base.csv")
y_train = train_base_df["target"]
oof_df = train_base_df
models_score_df = pd.DataFrame()
# model_names = []
test_preds_df = pd.DataFrame()

# 各モデルの読み込みと評価関数での精度
 - Model 1,2はlgbmのアンサンブルモデル、Model 3はTabularAutoMLのdenselightアルゴリズムを使ったnnモデル
 - 各モデルとその訓練に使ったデータを呼び出し、評価関数（上のgini_stability）での精度をそれぞれ出している
 - 各Modelのパラグラフで訓練データの予測値もモデル訓練時に保存したファイルから読み込んでおり、その値はoof_dfに保存している
 - テストデータに対する予測値はtest_preds_dfに保存し、その後のブレンディングの際に使っている
 - 要は
   - 評価関数（gini_stability関数）で訓練データで計算したスコア：model_score_df
   - 訓練データの予測値：oof_df
   - テストデータの予測値：test_preds_df

## Model 1

In [7]:
model_name = "model_1"
model_1 = joblib.load("/kaggle/input/home-credit-baseline-training/oof_model_1.pkl")
model_1
# model_names.append(model_name)

VotingClassifier(estimators=[('0',
                              LGBMClassifier(colsample_bynode=0.8,
                                             colsample_bytree=0.8, device='gpu',
                                             learning_rate=0.05, max_bin=255,
                                             max_depth=8, metric='auc',
                                             n_estimators=1000,
                                             objective='binary',
                                             random_state=42, verbose=-1)),
                             ('1',
                              LGBMClassifier(colsample_bynode=0.8,
                                             colsample_bytree=0.8, device='gpu',
                                             learning_rate=0.05, max_bin=255,
                                             max_depth=8, metric...
                                             colsample_bytree=0.8, device='gpu',
                                             learning_rate=0.05, max_bin=255,
                                             max_depth=8, metric='auc',
                                             n_estimators=1000,
                                             objective='binary',
                                             random_state=42, verbose=-1)),
                             ('4',
                              LGBMClassifier(colsample_bynode=0.8,
                                             colsample_bytree=0.8, device='gpu',
                                             learning_rate=0.05, max_bin=255,
                                             max_depth=8, metric='auc',
                                             n_estimators=1000,
                                             objective='binary',
                                             random_state=42, verbose=-1))],
                 voting='soft')

In [8]:
train_cols, cat_cols, drop_cols = joblib.load("/kaggle/input/home-credit-baseline-training/train_cat_columns.pkl")
print("train_cols:\t", len(train_cols))
print("cat_cols:\t", len(cat_cols))
print("drop_cols:\t", len(drop_cols))

train_cols:	 516
cat_cols:	 138
drop_cols:	 0


In [9]:
test_df = data_nb.prepare_df(data_nb.CFG.test_dir, cat_cols=cat_cols, mode="test", train_cols=train_cols)
display(test_df)

Feature engeneering...
  feats_df shape:	 (10, 926)
Filter cols...
  feats_df shape:	 (10, 515)
Convert to pandas...


,case_id,WEEK_NUM,month_decision,weekday_decision,assignmentdate_238D,assignmentdate_4527235D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtcount_4527229L,pmtcount_693L,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,secondquarter_766L,thirdquarter_1082L,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,credamount_770A,credtype_322L,currdebt_22A,currdebtcredtyperange_828A,datefirstoffer_1144D,datelastinstal40dpd_247D,datelastunpaid_3546854D,daysoverduetolerancedd_3976961L,deferredmnthsnum_166L,disbursedcredamount_1113A,disbursementtype_67L,downpmt_116A,dtlastpmtallstes_4499206D,eir_270L,equalitydataagreement_891L,firstclxcampaign_1125D,firstdatedue_489D,homephncnt_628L,inittransactionamount_650A,inittransactioncode_186L,interestrate_311L,isbidproduct_1095L,isdebitcard_729L,lastactivateddate_801D,lastapplicationdate_877D,lastapprcommoditycat_1041M,lastapprcredamount_781A,lastapprdate_640D,lastcancelreason_561M,lastdelinqdate_224D,lastrejectcommoditycat_161M,lastrejectcommodtypec_5251769M,lastrejectcredamount_222A,lastrejectdate_50D,lastrejectreason_759M,lastrejectreasonclient_4145040M,lastst_736L,maininc_215A,mastercontrelectronic_519L,mastercontrexist_109L,maxannuity_159A,maxdbddpdlast1m_3658939P,maxdbddpdtollast12m_3658940P,maxdbddpdtollast6m_4187119P,maxdebt4_972A,maxdpdfrom6mto36m_3546853P,maxdpdinstldate_3546855D,maxdpdinstlnum_3546846P,maxdpdlast12m_727P,maxdpdlast24m_143P,maxdpdlast3m_392P,maxdpdlast6m_474P,maxdpdlast9m_1059P,maxdpdtolerance_374P,maxinstallast24m_3658928A,maxlnamtstart6m_4525199A,maxoutstandbalancel12m_4187113A,maxpmtlast3m_4525190A,mindbddpdlast24m_3658935P,mindbdtollast24m_4525191P,mobilephncnt_593L,monthsannuity_845L,numactivecreds_622L,numactivecredschannel_414L,numactiverelcontr_750L,numcontrs3months_479L,numincomingpmts_3546848L,numinstlallpaidearly3d_817L,numinstls_657L,numinstlsallpaid_934L,numinstlswithdpd10_728L,numinstlswithdpd5_4187116L,numinstlswithoutdpd_562L,numinstmatpaidtearly2d_4499204L,numinstpaid_4499208L,numinstpaidearly3d_3546850L,numinstpaidearly3dest_4493216L,numinstpaidearly5d_1087L,numinstpaidearly5dest_4493211L,numinstpaidearly5dobd_4499205L,numinstpaidearly_338L,numinstpaidearlyest_4493214L,numinstpaidlastcontr_4325080L,numinstpaidlate1d_3546852L,numinstregularpaid_973L,numinstregularpaidest_4493210L,numinsttopaygr_769L,numinsttopaygrest_4493213L,numinstunpaidmax_3546851L,numinstunpaidmaxest_4493212L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,opencred_647L,paytype1st_925L,paytype_783L,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,type

In [10]:
test_preds_df['case_id'] = test_df['case_id']
test_preds_df.set_index('case_id', inplace=True)

In [11]:
X_test = test_df.drop(columns=["WEEK_NUM"] + drop_cols)
X_test = X_test.set_index("case_id")
print("X_test shape: ", X_test.shape)

y_pred_1 = pd.Series(predict_proba_in_batches(model_1, X_test), index=X_test.index)
test_preds_df[f"pred_{model_name}"] = y_pred_1

X_test shape:  (10, 513)
Processing batch: 1/1


In [12]:
oof_df[f"pred_{model_name}"] = joblib.load("/kaggle/input/home-credit-baseline-training/oof_pred.pkl")

In [13]:
gini_score = gini_stability(oof_df, score_col=f"pred_{model_name}")
models_score_df.loc[model_name, ["gini_score"]] = gini_score
print("gini_score:\t", gini_score)

gini_score:	 0.6497821787794288


## Model 2

In [14]:
model_name = "model_2"
model_2 = joblib.load("/kaggle/input/home-credit-baseline-training-model-2/oof_model.pkl")
model_2

VotingClassifier(estimators=[('0',
                              LGBMClassifier(colsample_bynode=0.8,
                                             colsample_bytree=0.8, device='gpu',
                                             learning_rate=0.05, max_bin=128,
                                             max_depth=8, metric='auc',
                                             n_estimators=1000,
                                             objective='binary',
                                             random_state=42, verbose=-1)),
                             ('1',
                              LGBMClassifier(colsample_bynode=0.8,
                                             colsample_bytree=0.8, device='gpu',
                                             learning_rate=0.05, max_bin=128,
                                             max_depth=8, metric...
                                             colsample_bytree=0.8, device='gpu',
                                             learning_rate=0.05, max_bin=128,
                                             max_depth=8, metric='auc',
                                             n_estimators=1000,
                                             objective='binary',
                                             random_state=42, verbose=-1)),
                             ('4',
                              LGBMClassifier(colsample_bynode=0.8,
                                             colsample_bytree=0.8, device='gpu',
                                             learning_rate=0.05, max_bin=128,
                                             max_depth=8, metric='auc',
                                             n_estimators=1000,
                                             objective='binary',
                                             random_state=42, verbose=-1))],
                 voting='soft')

In [15]:
train_cols, cat_cols, drop_cols = joblib.load("/kaggle/input/home-credit-baseline-training-model-2/train_cat_columns.pkl")
print("train_cols:\t", len(train_cols))
print("cat_cols:\t", len(cat_cols))
print("drop_cols:\t", len(drop_cols))

train_cols:	 516
cat_cols:	 138
drop_cols:	 89


In [16]:
X_test = test_df.drop(columns=["WEEK_NUM"] + drop_cols)
X_test = X_test.set_index("case_id")
print("X_test shape: ", X_test.shape)

y_pred_2 = pd.Series(predict_proba_in_batches(model_2, X_test), index=X_test.index)
test_preds_df[f"pred_{model_name}"] = y_pred_2

X_test shape:  (10, 424)
Processing batch: 1/1


In [17]:
oof_df[f"pred_{model_name}"] = joblib.load("/kaggle/input/home-credit-baseline-training-model-2/oof_pred.pkl")

In [18]:
gini_score = gini_stability(oof_df, score_col=f"pred_{model_name}")
models_score_df.loc[model_name, ["gini_score"]] = gini_score
print("gini_score:\t", gini_score)

gini_score:	 0.6430261760598828


## Model 3

In [19]:
!pip install --no-index -Uq --find-links=/kaggle/input/lightautoml-038-dependencies lightautoml==0.3.8

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.0.11 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2024.1.0 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2024.1.0 which is incompatible.
d

In [20]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

In [21]:
model_name = "denselight_model"
model_3 = joblib.load("/kaggle/input/home-credit-baseline-training-lightautoml/denselight_model.pkl")
model_3

In [22]:
train_cols, cat_cols, drop_cols = joblib.load("/kaggle/input/home-credit-baseline-training-lightautoml/train_cat_columns.pkl")
print("train_cols:\t", len(train_cols))
print("cat_cols:\t", len(cat_cols))
print("drop_cols:\t", len(drop_cols))

train_cols:	 418
cat_cols:	 113
drop_cols:	 98


In [23]:
X_test = test_df.drop(columns=["WEEK_NUM"] + drop_cols)
X_test = X_test.set_index("case_id")
print("X_test shape: ", X_test.shape)

y_pred_3 = pd.Series(
    predict_proba_in_batches(model_3, X_test, predict_mode = "lightautoml"),
    index=X_test.index)
test_preds_df[f"pred_{model_name}"] = y_pred_3

X_test shape:  (10, 415)
Processing batch: 1/1


In [24]:
oof_df[f"pred_{model_name}"] = joblib.load("/kaggle/input/home-credit-baseline-training-lightautoml/denselight_oof_preds.pkl")

In [25]:
gini_score = gini_stability(oof_df, score_col=f"pred_{model_name}")
models_score_df.loc[model_name, ["gini_score"]] = gini_score
print("gini_score:\t", gini_score)

gini_score:	 0.6360602763237749


## Estimation Results

In [26]:
models_score_df

,gini_score
model_1,0.649782
model_2,0.643026
denselight_model,0.636060


In [27]:
oof_df

,case_id,date_decision,MONTH,WEEK_NUM,target,pred_model_1,pred_model_2,pred_denselight_model
0,0,2019-01-03,201901,0,0,0.038914,0.036874,0.043001
1,1,2019-01-03,201901,0,0,0.060689,0.096309,0.028896
2,2,2019-01-04,201901,0,0,0.070097,0.083698,0.043408
3,3,2019-01-03,201901,0,0,0.078844,0.033708,0.056375
4,4,2019-01-04,201901,0,1,0.138104,0.120848,0.071871
...,...,...,...,...,...,...,...,...
1526654,2703450,2020-10-05,202010,91,0,0.000784,0.002075,0.000824
1526655,2703451,2020-10-05,202010,91,0,0.001888,0.002943,0.002522
1526656,2703452,2020-10-05,202010,91,0,0.041933,0.044443,0.055235
1526657,2703453,2020-10-05,202010,91,0,0.003564,0.002797,0.001453


In [28]:
del test_df
gc.collect()

0

# Blending

In [29]:
# 実際に評価スコアを呼び出す関数
def gini_wrapper(base_df):
    base_df = base_df[["WEEK_NUM", "target"]].copy()
    def gini_wrapper_inner(target, scores):
        base_df["score"] = scores
        gini_score = gini_stability(base_df, score_col="score")
        return 1 - gini_score
    return gini_wrapper_inner

### Hill climbing using minimize

In [30]:
# 上記のgini_wrapper関数とセットで重み付けを行うためのクラス
# この後にfind_weights関数で訓練データを使って重みを探索する
# これを使えばいい感じの重みを得られるくらいの理解で済ましたい
class WeightsSearcher:
    def __init__(self, loss_fn, bounds=[], mode="min", method='SLSQP'):
        self.loss_fn = loss_fn
        self.bounds = bounds
        self.mode = mode
        self.method = method # Nelder-Mead - for not smooth functions

    # これがgini_wrapper_innerの役割をしている
    # 予測値と重みを掛け合わせ、その結果から損失関数（ここでは1-gini_score）を計算する
    def _objective_function_wrapper(self, pred_values, true_targets, obj_fn):
        def objective_function(weights):
            pred_weighted = (pred_values * weights).sum(axis=1)
            score = obj_fn(true_targets, pred_weighted)
            return score
        return objective_function
    
    def find_weights(self, val_preds, true_targets):
        len_models = len(self.bounds)
        bounds = [0,1] * len_models if len(self.bounds) == 0 else self.bounds
        initial_weights = np.ones(len_models) / len_models
        objective_function = self._objective_function_wrapper(val_preds, true_targets, self.loss_fn)
        # 損失関数が最小になる（=gini_scoreが最大になる）重みを探す
        result = minimize(
            objective_function, 
            initial_weights, 
            bounds=bounds, 
            method=self.method,
        )
        optimized_weights = result.x
        optimized_weights /= np.sum(optimized_weights)
        return optimized_weights

In [31]:
model_names = models_score_df.index.to_list()
pred_cols = [f"pred_{name}" for name in model_names]
model_names

['model_1', 'model_2', 'denselight_model']

In [32]:
bounds = [(0, 1)] * len(pred_cols)
roc_auc_fn = lambda y_true, y_pred: 1 - roc_auc_score(y_true, y_pred)
gini_score_fn = gini_wrapper(oof_df)
w_searcher = WeightsSearcher(gini_score_fn, bounds, method='Nelder-Mead') # log_loss, gini_stability roc_auc_fn
optimized_weights = w_searcher.find_weights(
    oof_df[pred_cols].to_numpy(), 
    y_train
)
optimized_weights_df = pd.DataFrame(zip(model_names, optimized_weights), columns=['model', 'weight'])
display(optimized_weights_df)
print("sum: ", np.sum(optimized_weights))

,model,weight
0,model_1,0.479296
1,model_2,0.266185
2,denselight_model,0.254519


sum:  1.0


In [33]:
# 上記で算出した重みでaucを重み付けし、評価スコアを得る
oof_pred_optimized = (oof_df[pred_cols] * optimized_weights).sum(axis=1).to_numpy()
oof_df["pred_optimized"] = oof_pred_optimized # 重み付したaucを格納するカラムを作成
roc_auc_oof = roc_auc_score(y_train, oof_pred_optimized)
gini_score = gini_stability(oof_df, score_col="pred_optimized") # 重み付けしたaucをgini_stabilityに渡してスコアを得る
print("CV roc_auc_oof optimized:\t", roc_auc_oof)
print("CV gini_score:\t\t\t", gini_score)

CV roc_auc_oof optimized:	 0.8372383516308529
CV gini_score:			 0.6550665681333759


In [34]:
# テストデータのaucも重み付けしたバージョンを作成する
y_pred_all = (optimized_weights * test_preds_df[pred_cols]).sum(axis=1).to_numpy()
y_pred_all[:10]

array([0.15268928, 0.10499393, 0.08034035, 0.04036048, 0.03656391,
       0.03883912, 0.06013255, 0.04124608, 0.0480157 , 0.05783407])

# Submission

In [35]:
subm_df = pd.read_csv(data_nb.CFG.root_dir / "sample_submission.csv")
subm_df = subm_df.set_index("case_id")
subm_df["score"] = y_pred_all
display(subm_df.head())
print("Check null: ", subm_df["score"].isnull().any())

,score
case_id,
57543,0.152689
57549,0.104994
57551,0.080340
57552,0.040360
57569,0.036564


Check null:  False


In [36]:
subm_df.to_csv("submission.csv")